In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
from nemosis import static_table, dynamic_data_compiler
import plotly.express as px

raw_data_cache = '/Volumes/T7/NEMO_data'

In [2]:
# Get the volume bids for the energy market.
dispatch_load = dynamic_data_compiler(start_time='2021/03/01 00:00:00',
                                   end_time='2021/03/10 00:00:00',
                                   table_name='DISPATCHLOAD',
                                   raw_data_location=raw_data_cache)

INFO: Compiling data for table DISPATCHLOAD
INFO: Downloading data for table DISPATCHLOAD, year 2021, month 02
INFO: Creating feather file for DISPATCHLOAD, 2021, 02
INFO: Downloading data for table DISPATCHLOAD, year 2021, month 03
INFO: Creating feather file for DISPATCHLOAD, 2021, 03
INFO: Returning DISPATCHLOAD.


In [3]:
dispatch_load

,SETTLEMENTDATE,DUID,INTERVENTION,DISPATCHMODE,AGCSTATUS,INITIALMW,TOTALCLEARED,RAMPDOWNRATE,RAMPUPRATE,LOWER5MIN,...,RAISE60SEC,RAISE6SEC,LOWERREG,RAISEREG,AVAILABILITY,RAISEREGENABLEMENTMAX,RAISEREGENABLEMENTMIN,LOWERREGENABLEMENTMAX,LOWERREGENABLEMENTMIN,SEMIDISPATCHCAP
0,2021-03-01 00:05:00,AGLHAL,0,0,0,0.00000,0.00000,720.00,720.00,0.0,...,0.0000,0.0000,0.0,0.0,175.00000,0.0,0.0,0.0,0.0,0
1,2021-03-01 00:05:00,AGLSOM,0,0,0,0.00000,0.00000,480.00,480.00,0.0,...,0.0000,0.0000,0.0,0.0,160.00000,0.0,0.0,0.0,0.0,0
2,2021-03-01 00:05:00,ANGAST1,0,0,0,0.00000,0.00000,840.00,840.00,0.0,...,0.0000,0.0000,0.0,0.0,40.00000,0.0,0.0,0.0,0.0,0
3,2021-03-01 00:05:00,APD01,0,0,0,0.00000,0.00000,0.00,0.00,0.0,...,0.0001,0.0001,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0
4,2021-03-01 00:05:00,ARWF1,0,0,0,162.50000,161.88836,600.00,1200.00,0.0,...,0.0000,0.0000,0.0,0.0,161.88836,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
914971,2021-03-10 00:00:00,YENDWF1,0,0,0,12.41000,12.02600,1680.00,1680.00,0.0,...,0.0000,0.0000,0.0,0.0,12.02600,0.0,0.0,0.0,0.0,0
914972,2021-03-10 00:00:00,YWPS1,0,0,1,379.44809,380.00000,180.00,180.00,0.0,...,0.0000,0.0000,0.0,0.0,380.00000,365.0,250.0,365.0,250.0,0
914973,2021-03-10 00:00:00,YWPS2,0,0,1,338.84558,340.00000,180.00,180.00,10.0,...,0.0000,0.0000,0.0,0.0,340.00000,355.0,250.0,355.0,250.0,0
914974,2021-03-10 00:00:00,YWPS3,0,0,1,372.93799,375.00000,172.13,172.13,15.0,...,0.0000,0.0000,0.0,0.0,375.00000,385.0,250.0,385.0,250.0,0


In [4]:
# Group by DUID and sum the TOTALCLEARED column
generation_ranking = dispatch_load.groupby('DUID')['TOTALCLEARED'].sum().sort_values(ascending=False)

# Convert to dataframe and add rank column
generation_ranking = generation_ranking.reset_index()
generation_ranking['rank'] = generation_ranking.index + 1

# Show top 10
print("Top 10 generators by total dispatch:")
print(generation_ranking.head(10))

# Optional: Save full ranking to CSV
# generation_ranking.to_csv('generator_ranking.csv', index=False)

Top 10 generators by total dispatch:
     DUID  TOTALCLEARED  rank
0   KPP_1  1.866240e+06     1
1  LOYYB1  1.418682e+06     2
2  LOYYB2  1.385271e+06     3
3     MP1  1.360302e+06     4
4     MP2  1.339131e+06     5
5    LYA3  1.336849e+06     6
6    LYA1  1.336133e+06     7
7    LYA4  1.329320e+06     8
8    BW03  1.327687e+06     9
9    LYA2  1.305249e+06    10


In [5]:
# Download the latest Generators and Scheduled Loads table. The 
# update_static_file=True argument forces nemosis to download a new copy of 
# file from AEMO even if a copy already exists in the cache.
dispatch_units = static_table(table_name='Generators and Scheduled Loads', 
                              raw_data_location=raw_data_cache,
                              update_static_file=False)


INFO: Retrieving static table Generators and Scheduled Loads


In [6]:
# First get our ranking as before
generation_ranking = dispatch_load.groupby('DUID')['TOTALCLEARED'].sum().sort_values(ascending=False)
generation_ranking = generation_ranking.reset_index()
generation_ranking['rank'] = generation_ranking.index + 1

# Merge with dispatch_units table to get additional information
generator_details = generation_ranking.merge(
    dispatch_units[['DUID', 'Station Name', 'Region', 'Fuel Source - Primary', 'Fuel Source - Descriptor']], 
    on='DUID',
    how='left'
)

# Show top 10 with details
print("Top 10 generators by total dispatch with details:")
print(generator_details.head(10).to_string())

Top 10 generators by total dispatch with details:
     DUID  TOTALCLEARED  rank               Station Name Region Fuel Source - Primary Fuel Source - Descriptor
0   KPP_1  1.866240e+06     1  Kogan Creek Power Station   QLD1                Fossil               Black Coal
1  LOYYB1  1.418682e+06     2   Loy Yang B Power Station   VIC1                Fossil               Brown Coal
2  LOYYB2  1.385271e+06     3   Loy Yang B Power Station   VIC1                Fossil               Brown Coal
3     MP1  1.360302e+06     4     Mt Piper Power Station   NSW1                Fossil               Black Coal
4     MP2  1.339131e+06     5     Mt Piper Power Station   NSW1                Fossil               Black Coal
5    LYA3  1.336849e+06     6   Loy Yang A Power Station   VIC1                Fossil               Brown Coal
6    LYA1  1.336133e+06     7   Loy Yang A Power Station   VIC1                Fossil               Brown Coal
7    LYA4  1.329320e+06     8   Loy Yang A Power Station   VIC

Black coal:

Brown coal:
Total Marginal Cost For most brown coal generators, the total marginal cost typically ranges from $5-12/MWh, excluding fixed costs and carbon pricing3.